In [0]:
#  TASK -1 Create Delta Tables Using 3 Methods
#1.1. Load the sales_data.csv file into a DataFrame.
dbutils.fs.cp("file:/Workspace/Shared/sales_data.csv", "dbfs:/FileStore/sales_data.csv")
df_sales=spark.read.format("csv").option ("header", "true").load("/FileStore/sales_data.csv")



In [0]:
#1.2. Write the DataFrame as a Delta Table.
df_sales.write.format("delta").mode("overwrite").save("/delta/sales_data")
df_sales=spark.read.format("delta").load("/delta/sales_data")

In [0]:
#1.3. Load the customer_data.json file into a DataFrame.
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
schema=StructType([  
                   StructField("CustomerID", StringType(), True),
                   StructField("CustomerName", StringType(), True),
                   StructField("Region", StringType(), True),
                   StructField("SignupDate", StringType(), True) 
])

dbutils.fs.cp("file:/Workspace/Shared/customer_data.json", "dbfs:/FileStore/customer_data.json")

df_customer=spark.read.format("json").schema(schema).load("dbfs:/FileStore/customer_data.json") 
df_customer.show()



+----------+-------------+------+----------+
|CustomerID| CustomerName|Region|SignupDate|
+----------+-------------+------+----------+
|      C001|     John Doe| North|2022-07-01|
|      C002|   Jane Smith| South|2023-02-15|
|      C003|Emily Johnson|  East|2021-11-20|
|      C004|Michael Brown|  West|2022-12-05|
|      C005|  Linda Davis| North|2023-03-10|
+----------+-------------+------+----------+



In [0]:
#1.4. Write the DataFrame as a Delta Table.
df_customer.createOrReplaceTempView("customer_view")

spark.sql("""
CREATE TABLE IF NOT EXISTS delta_customer
USING DELTA
AS SELECT * FROM customer_view
""")

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
#1.5 Load the Parquet file
parquet_df = spark.read.parquet("/FileStore/sample_parquet_file.parquet")

# Convert the Parquet file to a Delta Table
parquet_df.write.format("delta").mode("overwrite").save("/delta/sample_parquet_to_delta")

In [0]:
# TASK-2 Data Management
#2.1. Load the new_sales_data.csv file into a DataFrame.
dbutils.fs.cp("file:/Workspace/Shared/new_sales_data.csv", "dbfs:/FileStore/new_sales_data.csv")
df_new_sales=spark.read.format("csv").option ("header", "true").load("/FileStore/new_sales_data.csv")
df_new_sales.show()



+-------+----------+----------+--------+--------+-----+
|OrderID| OrderDate|CustomerID| Product|Quantity|Price|
+-------+----------+----------+--------+--------+-----+
|   1009|2024-01-22|      C006|Widget E|      14|20.00|
|   1010|2024-01-23|      C007|Widget F|       6|35.00|
|   1002|2024-01-16|      C002|Widget B|      10|15.75|
+-------+----------+----------+--------+--------+-----+



In [0]:
#2.2. Write the new DataFrame as a Delta Table.
df_new_sales.write.format("delta").mode("overwrite").save("/delta/new_sales_data")
df_new_sales=spark.read.format("delta").load("/delta/new_sales_data")
df_sales.createOrReplaceTempView("delta_sales") 
df_new_sales.createOrReplaceTempView("new_sales")

In [0]:
#2.3. Perform a MERGE INTO operation to update and insert records into the existing Delta table.
spark.sql("""
          MERGE INTO delta_sales AS target 
          USING new_sales AS source 
          ON target.OrderID = source.OrderID 
          WHEN MATCHED THEN UPDATE SET target.OrderDate = source.OrderDate,target.CustomerID = source.CustomerID,target.Product = source.Product, target.Quantity = source.Quantity, target.Price = source.Price
          WHEN NOT MATCHED THEN INSERT (OrderID, OrderDate, CustomerID, Product, Quantity, Price)
          VALUES (source.OrderID, source.OrderDate, source.CustomerID, source.Product, source.Quantity, source.Price)
""")
spark.sql("SELECT * FROM delta_sales").show()

+-------+----------+----------+--------+--------+-----+
|OrderID| OrderDate|CustomerID| Product|Quantity|Price|
+-------+----------+----------+--------+--------+-----+
|   1001|2024-01-15|      C001|Widget A|      10|25.50|
|   1003|2024-01-16|      C001|Widget C|       8|22.50|
|   1004|2024-01-17|      C003|Widget A|      15|25.50|
|   1005|2024-01-18|      C004|Widget D|       7|30.00|
|   1006|2024-01-19|      C002|Widget B|       9|15.75|
|   1007|2024-01-20|      C005|Widget C|      12|22.50|
|   1008|2024-01-21|      C003|Widget A|      10|25.50|
|   1002|2024-01-16|      C002|Widget B|      10|15.75|
|   1009|2024-01-22|      C006|Widget E|      14|20.00|
|   1010|2024-01-23|      C007|Widget F|       6|35.00|
+-------+----------+----------+--------+--------+-----+



In [0]:
# Task - 3 Optimize Delta Table
spark.sql("CREATE TABLE IF NOT EXISTS delta_sales_table USING DELTA LOCATION '/delta/sales_data'")
#3.1. Apply the OPTIMIZE command on the Delta Table and use Z-Ordering on an appropriate column.
spark.sql("""
	OPTIMIZE delta_customer ZORDER BY CustomerID
""")

spark.sql("""
	OPTIMIZE delta_sales_table ZORDER BY Product
""")

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,clusteringStats:struct<inputZCubeFiles:struct<numFiles:bigint,size:bigint>,inputOtherFiles:struct<numFiles:bigint,size:bigint>,inputNumZCubes:bigint,mergedFiles:struct<numFiles:bigint,size:bigint>,numOutputZCubes:bigint>,numBins:bigint,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,

In [0]:
# TASK - 4 Advanced Features
#4.1. Use DESCRIBE HISTORY to inspect the history of changes for a Delta Table.
spark.sql("DESCRIBE HISTORY delta_customer").show(truncate=False)

spark.sql("DESCRIBE HISTORY delta_sales_table").show(truncate=False)



+-------+-------------------+----------------+----------------------------------+----------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------+----+------------------+--------------------+-----------+-----------------+-------------+-----------------------------------------------------------+------------+------------------------------------------+
|version|timestamp          |userId          |userName                          |operation             |operationParameters                                                                                                                                     |job |notebook          |clusterId           |readVersion|isolationLevel   |isBlindAppend|operationMetrics                                           |userMetadata|engineInfo                                |
+-------+-------------------+----------------+----------------------------

In [0]:
#4.2. Use VACUUM to remove old files from the Delta Table.
spark.sql("""
	VACUUM delta_customer RETAIN 168 HOURS
""")

spark.sql("""
	VACUUM delta_sales_table RETAIN 168 HOURS
""")

DataFrame[path: string]

In [0]:
# Task-5  Hands-on Exercises
#5.1. Using Delta Lake for Data Versioning:
df_version = spark.read.format("delta").option("versionAsOf", 0).table("delta_sales_table")
df_version.show()



+-------+----------+----------+--------+--------+-----+
|OrderID| OrderDate|CustomerID| Product|Quantity|Price|
+-------+----------+----------+--------+--------+-----+
|   1001|2024-01-15|      C001|Widget A|      10|25.50|
|   1002|2024-01-16|      C002|Widget B|       5|15.75|
|   1003|2024-01-16|      C001|Widget C|       8|22.50|
|   1004|2024-01-17|      C003|Widget A|      15|25.50|
|   1005|2024-01-18|      C004|Widget D|       7|30.00|
|   1006|2024-01-19|      C002|Widget B|       9|15.75|
|   1007|2024-01-20|      C005|Widget C|      12|22.50|
|   1008|2024-01-21|      C003|Widget A|      10|25.50|
+-------+----------+----------+--------+--------+-----+



In [0]:
#5.2. Building a Reliable Data Lake with Delta Lake:
#Implement schema enforcement and handle data updates with Delta Lake.
#schema enforcement
from pyspark.sql.types import StructType, StructField,IntegerType, StringType, DoubleType

schema = StructType([
    StructField("OrderID", IntegerType(), nullable=False),
    StructField("OrderDate", StringType(), nullable=False),
    StructField("CustomerID", StringType(), nullable=False),
    StructField("Product", StringType(), nullable=False),
    StructField("Quantity", IntegerType(), nullable=False),
    StructField("Price", DoubleType(), nullable=False)
])

df = spark.read.csv("dbfs:/FileStore/sales_data.csv", schema=schema, header=True)
df.write.format("delta").mode("overwrite").save("/delta/delta_sales_table")

#handle data updates
spark.sql("""
          MERGE INTO delta_sales AS target
          USING new_sales AS source
          ON target.OrderID = source.OrderID
          WHEN MATCHED THEN UPDATE SET target.OrderDate = source.OrderDate,target.CustomerID = source.CustomerID,target.Product = source.Product, target.Quantity = source.Quantity, target.Price = source.Price
          WHEN NOT MATCHED THEN INSERT (OrderID, OrderDate, CustomerID, Product, Quantity, Price)
          VALUES (source.OrderID, source.OrderDate, source.CustomerID, source.Product, source.Quantity, source.Price)
""")
spark.sql("SELECT * FROM delta_sales").show()


#Optimize data layout and perform vacuum operations to maintain storage efficiency
spark.sql("""
	OPTIMIZE delta_customer ZORDER BY CustomerID
""")
spark.sql("""
	VACUUM delta_customer RETAIN 168 HOURS
""")


+-------+----------+----------+--------+--------+-----+
|OrderID| OrderDate|CustomerID| Product|Quantity|Price|
+-------+----------+----------+--------+--------+-----+
|   1001|2024-01-15|      C001|Widget A|      10|25.50|
|   1003|2024-01-16|      C001|Widget C|       8|22.50|
|   1004|2024-01-17|      C003|Widget A|      15|25.50|
|   1005|2024-01-18|      C004|Widget D|       7|30.00|
|   1006|2024-01-19|      C002|Widget B|       9|15.75|
|   1007|2024-01-20|      C005|Widget C|      12|22.50|
|   1008|2024-01-21|      C003|Widget A|      10|25.50|
|   1009|2024-01-22|      C006|Widget E|      14|20.00|
|   1010|2024-01-23|      C007|Widget F|       6|35.00|
|   1002|2024-01-16|      C002|Widget B|      10|15.75|
+-------+----------+----------+--------+--------+-----+



DataFrame[path: string]